<a href="https://colab.research.google.com/github/IA-Programming/phixtral-inference-into-a-free-colab-8bit/blob/main/Phixtral_Inference_8_bit_using_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Thu Jan 25 14:30:06 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# @title # Phixtral inference

# @markdown Quick notebook to run [mlabonne/phixtral-2x2_8](https://huggingface.co/mlabonne/phixtral-2x2_8) (recommended) and [mlabonne/phixtral-4x2_8](https://huggingface.co/mlabonne/phixtral-4x2_8) on a free T4 GPU in 8-bit precision.

!pip install -qqq --upgrade transformers einops accelerate bitsandbytes --progress-bar off

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "phixtral-4x2_8" # @param ["phixtral-2x2_8", "phixtral-4x2_8"]
instruction = "Write an epic poem about Ancient Rome." # @param {type:"string"}

prompt = f'''
<|im_start|>system
You are Phixtral, a helpful AI assistant.<|im_end|>
<|im_start|>user
{instruction}<|im_end|>
<|im_start|>assistant
'''

torch.set_default_device("cuda")

# Load the model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    f"mlabonne/{model_name}",
    torch_dtype="auto",
    load_in_8bit=True,
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(
    f"mlabonne/{model_name}",
    trust_remote_code=True
)

# Tokenize the input string
inputs = tokenizer(
    prompt,
    return_tensors="pt",
    return_attention_mask=False
)

# Generate text using the model
outputs = model.generate(**inputs, max_length=200)

# Decode and print the output
text = tokenizer.batch_decode(outputs)[0]
print(text[len(prompt):])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/8.04k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.12M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/470 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In the heart of the Eternal City,
Where the sun kissed the cobblestone streets,
Lived a civilization of grandeur,
Ancient Rome, a tale of feats.

From the mighty legions that marched,
To the great emperors who ruled,
This land of history and lore,
Is a story that never grew.

In the shadow of the mighty Colosseum,
Where gladiators fought for fame,
And in the Forum, where senators spoke,
The pulse of Rome was never the same.

From the rise of Julius Caesar,
To the fall of the mighty Roman Empire,
This land of triumph and strife,
Is a tale that will never die.

The Roman Republic, a time of peace,
Where laws were made and


In [ ]:
# Converstion hystory
conversation_history = []

def chatbot(instruction, conversation_history=conversation_history):

  system_prompt = '''
<|im_start|>system
You are Phixtral, a helpful AI assistant.<|im_end|>
'''

  # Prompt to make it a chatbot
  prompt = '/n'.join([f"<|im_start|>user\n{i}<|im_end|>\n<|im_start|>assistant\n{o}<|im_end|>" for i, o in conversation_history])

  new_message = f'\n<|im_start|>user\n{instruction}<|im_end|>\n<|im_start|>assistant\n'

  prompt = system_prompt + prompt + new_message

  print('\033[1;31m' + '---'*100 + '\n' + prompt + '\n' + '---'*100 + '\033[0m')

  # Tokenize the input string
  inputs = tokenizer(
      prompt,
      return_tensors="pt",
      return_attention_mask=False
  )

  # Generate text using the model
  outputs = model.generate(**inputs, max_length=200)

  # Decode and print the output
  text = tokenizer.batch_decode(outputs)[0]
  try:
    respuesta = text[len(prompt):].split("<|im_end|>")[0]
  except:
    respuesta = text[len(prompt):]

  conversation_history.append((instruction, respuesta))

  return respuesta

In [ ]:
while True:
  question = input('>>>')
  if question == 'exit':
    break
  respuesta = chatbot(question)
  print('answer: ', respuesta)

>>>hi
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

<|im_start|>system
You are Phixtral, a helpful AI assistant.<|im_end|>

<|im_start|>user
hi<|im_end|>
<|im_start|>assistant

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
answer:  Hello! How can I assist you today?
>>>my name is joseph and what's yours?
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Input length of input_ids is 240, but `max_length` is set to 200. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


answer:  The
>>>exit
